# 步骤
1. 函数封装
2. API:打包为Pip/Conda
3. Django:网页服务

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests, time, re, math, openpyxl, datetime, os, shutil, psutil
from tqdm import *
import xlwings as xw
from selenium import webdriver
#from splinter import Browser
#from pandas.io.json import json_normalize

# Def get position function: 增加程序对数据源结构变化的稳定性
def f_p(df, feature):
    feature_list = df.iloc[:,0].get_values().tolist() # 第一列为特征名称，Index为连续自然数
    return [i for i,x in enumerate(feature_list) if x == feature][0]

# Def Check Nan Function
# https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-in-python
def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

# Get current file path for migration consistency
path = os.getcwd().replace('\\','/') #r'%s' % os.getcwd().replace('\\','/')

# 定义函数：从文本中根据数字类型提取Int或者Float数值
def find_number(text):
    if len(re.findall(r'\.',text)) == 0:
        return float(re.findall(r'\d+',text.replace(",", ""))[0])
    else:
        return float(re.findall(r'\d+\.\d+',text.replace(",", ""))[0])

In [2]:
keywords = ['个股期权','期权','场外期权','衍生品','杠杆','配资']

In [8]:
def get_accounts(keywords):
    df = pd.DataFrame(columns=['关键字','公众号名称','公众号网址','功能介绍','微信认证','最近文章'])

    # type1:公众号搜索；type2:文章搜索
    try:
        for keyword in tqdm(keywords):
            url = 'http://weixin.sogou.com/weixin?type=1&query='+keyword
            headers = {'User-Agent': UserAgent().random}
            r = requests.get(url,headers)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text,'lxml')
            page_numbers = len(soup.find('div',class_='p-fy',id='pagebar_container').find_all('a'))

            for p in range(page_numbers):
                url = 'http://weixin.sogou.com/weixin?type=1&page='+str(p)+'&query='+'个股期权'
                r = requests.get(url,headers)
                r.encoding = 'utf-8'
                soup = BeautifulSoup(r.text,'lxml')

                for i in range(len(soup.find_all('li',id=True))):
                    if len(soup.find_all('li',id=True)[i].find_all('dl')) == 1:
                        df = df.append({'关键字':keyword,\
                                        '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                        '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n','')\
                                       },ignore_index=True)
                    elif len(soup.find_all('li',id=True)[i].find_all('dl')) == 2:
                        df = df.append({'关键字':keyword,\
                                        '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                        '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n',''),\
                                        '最近文章':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('\n\n')[1].split('document')[0]\
                                       },ignore_index=True)
                    else:
                        df = df.append({'关键字':keyword,\
                                        '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                        '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n',''),\
                                        '微信认证':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('：\n')[1][:-1],\
                                        '最近文章':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('\n\n')[1].split('document')[0]\
                                       },ignore_index=True)
                time.sleep(5)
            time.sleep(5)
    except AttributeError as e:
        print ('访问过于频繁，请设置time.sleep()!', e)
    finally:
        return df

In [9]:
get_accounts(keywords) #keywords为list

 67%|██████▋   | 4/6 [03:53<01:56, 58.46s/it]

访问过于频繁，请设置time.sleep()! 'NoneType' object has no attribute 'find_all'


In [10]:
df

NameError: name 'df' is not defined

In [7]:
df.to_excel('期权公众号数据库.xlsx',encoding='gb18030')